In [1]:
import csv
import tkinter as tk
from tkinter import messagebox
import random
import unicodedata
from collections import Counter

class WordleGame:
    def __init__(self, file_path, word_length=5, max_attempts=6):
        self.word_length = word_length
        self.max_attempts = max_attempts
        self.words = self.load_words(file_path)
        self.target_word = random.choice(self.words)
        self.attempts = 0
        self.key_states = {}  # To track the state of each key
        self.selected_cell = (0, 0)  # Initialize to the first cell

        self.root = tk.Tk()
        self.root.title("Wordle Game")

        self.create_grid()
        self.create_keyboard()

        # Bind keyboard events
        self.root.bind('<Key>', self.on_keyboard_press)

    def load_words(self, file_path):
        words = []
        with open(file_path, 'r', encoding='utf-8') as file:
            reader = csv.reader(file, delimiter='\t')
            next(reader)  # Skip the header row
            for row in reader:
                word = row[0]  # Assuming the word is in the first column
                if len(word) == self.word_length:
                    normalized_word = self.normalize_word(word)
                    words.append(normalized_word)
        return words

    def normalize_word(self, word):
        # Supprimer les accents
        normalized = ''.join(
            c for c in unicodedata.normalize('NFD', word)
            if unicodedata.category(c) != 'Mn'
        )
        return normalized.lower()

    def create_grid(self):
        self.grid_frame = tk.Frame(self.root)
        self.grid_frame.pack()

        self.cells = []
        for i in range(self.max_attempts):
            row = []
            for j in range(self.word_length):
                cell = tk.Label(self.grid_frame, text='', width=3, height=2, borderwidth=1, relief='solid')
                cell.grid(row=i, column=j, padx=2, pady=2)
                cell.bind('<Button-1>', lambda e, r=i, c=j: self.select_cell(r, c))
                row.append(cell)
            self.cells.append(row)

    def create_keyboard(self):
        self.keyboard_frame = tk.Frame(self.root)
        self.keyboard_frame.pack()

        keys = 'azertyuiopqsdfghjklmwxcvbn'
        self.buttons = {}
        for i, key in enumerate(keys):
            button = tk.Button(self.keyboard_frame, text=key, width=3, height=2,
                               command=lambda k=key: self.on_key_press(k))
            button.grid(row=i // 10, column=i % 10, padx=2, pady=2)
            self.buttons[key] = button
            self.key_states[key] = 'default'  # Track key state

        # Ajouter les boutons "Effacer" et "Entrée"
        delete_button = tk.Button(self.keyboard_frame, text='Effacer', width=6, height=2, command=self.on_delete_press)
        delete_button.grid(row=3, column=8, columnspan=2, padx=2, pady=2)

        enter_button = tk.Button(self.keyboard_frame, text='Entrée', width=6, height=2, command=self.on_enter_press)
        enter_button.grid(row=3, column=10, columnspan=2, padx=2, pady=2)

    def select_cell(self, row, col):
        if self.attempts == row:
            self.selected_cell = (row, col)
            for i, cell in enumerate(self.cells[row]):
                if i == col:
                    cell.config(bg='lightblue')
                else:
                    cell.config(bg='SystemButtonFace')

    def on_key_press(self, key):
        if self.attempts < self.max_attempts:
            row, col = self.selected_cell
            if self.cells[row][col]['text'] == '':
                self.cells[row][col].config(text=key)
                if col + 1 < self.word_length:
                    self.selected_cell = (row, col + 1)
                self.select_cell(row, self.selected_cell[1])

    def on_delete_press(self):
        if self.attempts < self.max_attempts:
            row, col = self.selected_cell
            if col > 0 and self.cells[row][col - 1]['text'] != '':
                self.cells[row][col - 1].config(text='')
                self.selected_cell = (row, col - 1)
            elif col == 0 and self.cells[row][col]['text'] != '':
                self.cells[row][col].config(text='')
            self.select_cell(row, self.selected_cell[1])

    def on_enter_press(self):
        if self.attempts < self.max_attempts:
            current_row = self.cells[self.attempts]
            if all(cell['text'] != '' for cell in current_row):
                self.check_word()

    def on_keyboard_press(self, event):
        if event.keysym == 'BackSpace':
            self.on_delete_press()
        elif event.keysym == 'Return':
            self.on_enter_press()
        elif len(event.keysym) == 1 and event.keysym in 'azertyuiopqsdfghjklmwxcvbn':
            self.on_key_press(event.keysym)

    def check_word(self):
        current_word = ''.join(cell['text'] for cell in self.cells[self.attempts])
        normalized_current_word = self.normalize_word(current_word)
        if normalized_current_word not in self.words:
            messagebox.showerror("Mot invalide", "Ce mot n'existe pas. Essayez un autre mot.")
            return

        if normalized_current_word == self.target_word:
            for cell in self.cells[self.attempts]:
                cell.config(bg='green', fg='white')
            messagebox.showinfo("Félicitations", "Vous avez trouvé le mot !")
            self.root.quit()
        else:
            self.color_cells(normalized_current_word)
            self.attempts += 1
            if self.attempts < self.max_attempts:
                self.selected_cell = (self.attempts, 0)  # Move to the first cell of the new row
                self.select_cell(self.attempts, 0)
            else:
                messagebox.showinfo("Perdu", f"Le mot était : {self.target_word}")
                self.root.quit()

    def color_cells(self, word):
        target_counter = Counter(self.target_word)
        word_counter = Counter(word)

        # First pass: mark correct letters
        for i, letter in enumerate(word):
            if letter == self.target_word[i]:
                self.cells[self.attempts][i].config(bg='green', fg='white')
                target_counter[letter] -= 1
                word_counter[letter] -= 1
                if self.key_states[letter] != 'green':
                    self.buttons[letter].config(bg='green', fg='white')
                    self.key_states[letter] = 'green'

        # Second pass: mark present but misplaced letters
        for i, letter in enumerate(word):
            if letter != self.target_word[i] and word_counter[letter] > 0 and target_counter[letter] > 0:
                self.cells[self.attempts][i].config(bg='yellow', fg='black')
                target_counter[letter] -= 1
                word_counter[letter] -= 1
                if self.key_states[letter] != 'green':
                    self.buttons[letter].config(bg='yellow', fg='black')
                    self.key_states[letter] = 'yellow'

        # Third pass: mark absent letters
        for i, letter in enumerate(word):
            if self.cells[self.attempts][i]['bg'] == 'SystemButtonFace':
                self.cells[self.attempts][i].config(bg='gray', fg='black')
                if self.key_states[letter] == 'default':
                    self.buttons[letter].config(bg='gray', fg='black')
                    self.key_states[letter] = 'gray'

    def run(self):
        self.root.mainloop()

# Exemple d'utilisation
file_path = 'Lexique383.tsv'
game = WordleGame(file_path)
game.run()
